In [1]:
from preprocess import *
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
tf.enable_eager_execution()

# Second dimension of the feature is dim2
feature_dim_2 = 11

# # Feature dimension
feature_dim_1 = 20
channel = 1

verbose = 1
num_classes = 3

In [2]:
word=tuple(["bed", "cat","happy"])

In [3]:
def parse_func(s_example):
    features = {
                'mfcc2':tf.FixedLenFeature((20,11,1),tf.float32),
                'word': tf.FixedLenFeature((), tf.string)
                }
    example = tf.parse_single_example(s_example, features=features)
    return example

In [4]:
mapping_strings = tf.constant(["bed", "cat", "happy"])
table = tf.contrib.lookup.index_table_from_tensor(mapping=mapping_strings)

def map2hot_vector(feature):
    word_string = feature.pop('word')
    word_index = table.lookup(word_string)
    return (feature.pop('mfcc2'),tf.one_hot(word_index,3,dtype=tf.float32))


In [5]:
data = tf.data.Dataset.list_files('*.tfrecords').interleave(
        lambda filename:tf.data.TFRecordDataset(filename),cycle_length=3).\
        shuffle(2000).map(parse_func).map(map2hot_vector)
train_data = data.take(4000).repeat().batch(64)
val_data = data.skip(4000).repeat().batch(64)

In [6]:
def get_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(feature_dim_1, feature_dim_2, channel)))
    model.add(Conv2D(48, kernel_size=(2, 2), activation='relu'))
    model.add(Conv2D(120, kernel_size=(2, 2), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  #loss=keras.losses.categorical_crossentropy,
                  optimizer=tf.train.AdamOptimizer(),
                  #optimizer=tf.train.AdadeltaOptimizer(),
                  metrics=['accuracy'])
    return model

# Predicts one sample
def predict(filepath, model):
    sample = wav2mfcc(filepath)
    sample_reshaped = sample.reshape(1, feature_dim_1, feature_dim_2, channel)
    
    return word[np.argmax(model.predict(sample_reshaped))]

# Building The Model Then Training it

In [7]:
model = get_model()

model.fit(train_data,epochs=30, steps_per_epoch=64,verbose=verbose)

Epoch 1/30
64/64 [==============================] - 7s 109ms/step - loss: 1.3374 - acc: 0.4810
Epoch 2/30
64/64 [==============================] - 6s 99ms/step - loss: 0.5996 - acc: 0.7566
Epoch 3/30
64/64 [==============================] - 6s 98ms/step - loss: 0.4092 - acc: 0.8455
Epoch 4/30
64/64 [==============================] - 6s 94ms/step - loss: 0.3104 - acc: 0.8928
Epoch 5/30
64/64 [==============================] - 6s 90ms/step - loss: 0.2669 - acc: 0.9106
Epoch 6/30
64/64 [==============================] - 7s 106ms/step - loss: 0.2175 - acc: 0.9290
Epoch 7/30
64/64 [==============================] - 6s 99ms/step - loss: 0.1725 - acc: 0.9407
Epoch 8/30
64/64 [==============================] - 7s 105ms/step - loss: 0.1556 - acc: 0.9507
Epoch 9/30
64/64 [==============================] - 7s 102ms/step - loss: 0.1206 - acc: 0.9607
Epoch 10/30
64/64 [==============================] - 6s 87ms/step - loss: 0.1223 - acc: 0.9548
Epoch 11/30
64/64 [==============================] - 7s

## Prediction

In [8]:
model.evaluate(val_data,steps=20)

20/20 [==============================] - 1s 63ms/step


[0.003674410890380386, 1.0]

In [9]:
print(predict('./data/cat/1dce06e8_nohash_0.wav', model=model))
print(predict('./data/bed/01b4757a_nohash_0.wav', model=model))
print(predict('./data/bed/73f0c1f7_nohash_0.wav',model=model))
print(predict('./data/happy/0b77ee66_nohash_1.wav',model=model))

cat
bed
bed
happy


In [10]:
val_data2 = tf.data.TFRecordDataset('bed.tfrecords')\
        .map(parse_func).map(map2hot_vector).batch(1000)
model.evaluate(val_data2,steps=1)

1/1 [==============================] - 0s 318ms/step


[0.008689881302416325, 0.9980000257492065]

In [11]:
import IPython.display as ipd

In [12]:
ipd.Audio('./data/cat/0ab3b47d_nohash_1.wav')